<a href="https://colab.research.google.com/github/amcheyre-nw/nlp_chatbot/blob/main/nlp_chatbot_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/nlp

/content/gdrive/MyDrive/nlp


In [3]:
! git clone https://github.com/budzianowski/multiwoz

fatal: destination path 'multiwoz' already exists and is not an empty directory.


In [4]:
! git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [26]:
!pip install rouge_score
!pip install git+https://github.com/google-research/bleurt.git
!pip install sacrebleu

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-bnx8ralx
  Running command git clone -q https://github.com/google-research/bleurt.git /tmp/pip-req-build-bnx8ralx
     |████████████████████████████████| 90 kB 6.1 MB/s 


In [5]:
!pip install tokenizers
!pip install transformers
!pip install datasets

from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
from datasets import load_metric
from pprint import pprint

import torch
import math
import time
import sys
import json
import numpy as np

     |████████████████████████████████| 6.5 MB 8.2 MB/s 
     |████████████████████████████████| 3.8 MB 8.5 MB/s 
     |████████████████████████████████| 895 kB 68.4 MB/s 
     |████████████████████████████████| 596 kB 85.0 MB/s 
     |████████████████████████████████| 67 kB 7.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 8.4 MB/s 
     |████████████████████████████████| 212 kB 84.2 MB/s 
     |████████████████████████████████| 1.1 MB 78.6 MB/s 
     |████████████████████████████████| 134 kB 80.6 MB/s 
     |████████████████████████████████| 127 kB 85.7 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 97.2 MB/s 
     |████████████████████████████████| 271 kB 70.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3

In [6]:
def make_woz_datasets(bKnowledge):
    
    if bKnowledge:
        out_names = ['woz.train_c.txt','woz.valid_c.txt','woz.test_c.txt']
    else:
        out_names = ['woz.train_b.txt','woz.valid_a.txt','woz.test_a.txt','woz.valid_b.txt','woz.test_b.txt']
    max_ins = [18,2,2,2,2]
    
    count = 0
    counts = []
    for dataset in range(len(out_names)):
        fout = open(out_names[dataset],'wt')
        for dialog in range(1,max_ins[dataset],1):
            file_name = '/content/gdrive/MyDrive/nlp/multiwoz/train/dialogues_%03d.json' % dialog
            print(file_name)
            with open(file_name) as f:
                data = json.load(f)
            for dialogue in data:
                if len(dialogue['services']) == 1:
                    if dialogue['services'][0] == 'restaurant':
                        prev_speaker = ''
                        prev_utterance = ''
                        for turn in dialogue['turns']:
                            count = count + 1
                            speaker = turn['speaker']
                            utterance = turn['utterance']
                            
                            for frame in turn['frames']:
                                if frame['service'] == 'restaurant':
                                    knowledge = ''
                                    try:
                                        knowledge = '[KNOWLEDGE] '
                                        for slot in frame['slots']:
                                            temp = '%s [EQUAL] %s [SEP] ' % (slot['slot'],slot['value'])
                                            knowledge = knowledge + temp
                                    except:
                                        nothing = 1
                                    try:
                                        if len(knowledge) == 0:
                                            knowledge = '[KNOWLEDGE] '
                                        try:
                                            intent = frame['state']['active_intent']
                                            temp = '%s [EQUAL] %s [SEP] ' % ('active_intent',intent)
                                            knowledge = knowledge + temp
                                            slot_values = frame['state']['slot_values']
                                            for slot in slot_values:
                                                vals = slot_values[slot]
                                                for val in vals:
                                                    temp = '%s [EQUAL] %s [SEP] ' % (slot,val)
                                                    knowledge = knowledge + temp
                                        except:
                                            nothing = 1
                                    except:
                                        noting = 1
                            
                            if len(prev_speaker) > 0:
                                if not bKnowledge:
                                    knowledge = ''
                                if dataset == 0:
                                    text = '[%s] %s %s [%s] %s [END]' % (prev_speaker,
                                                                         prev_utterance,
                                                                         knowledge,
                                                                         speaker,
                                                                         utterance)
                                else:
                                    text = '[%s] %s %s [%s] | %s [END]' % (prev_speaker,
                                                                         prev_utterance,
                                                                         knowledge,
                                                                         speaker,
                                                                         utterance)
                                fout.write('%s\n' % (text))
                                print(text)
                            prev_speaker = speaker
                            prev_utterance = utterance
    counts.append(count)
    count = 0
    print(counts)

In [7]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2iwucywe
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-2iwucywe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3836842 sha256=cbec428c47d92fc5ef1e832b5606ba700303d0362016a0bfcd09973fa3626ab2
  Stored in directory: /tmp/pip-ephem-wheel-cache-a41hchfj/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.17.0
    Uninstalling transformers-4.17.0:
      Successfully uninstalled transformers-4.17.0


In [8]:
make_woz_datasets(True)
make_woz_datasets(False)

Streaming output truncated to the last 5000 lines.
[SYSTEM] Frankie and Bennys serves Italian food in the south part of town. Is there anything else I can help you with?  [USER] Yes, what is their phone number and where are they located at? [END]
[USER] Yes, what is their phone number and where are they located at?  [SYSTEM] Their address is Cambridge Leisure Park Clifton Way Cherry Hinton and phone number is 01223 412430 [END]
[SYSTEM] Their address is Cambridge Leisure Park Clifton Way Cherry Hinton and phone number is 01223 412430  [USER] Thank you goodbye. [END]
[USER] Thank you goodbye.  [SYSTEM] Thank you for using our system. Good bye [END]
[USER] I am looking for a restaurant that serves British food in the expensive price range.  [SYSTEM] Okay how about the Travellers Rest on the west side? [END]
[SYSTEM] Okay how about the Travellers Rest on the west side?  [USER] No. I need something in the centre. [END]
[USER] No. I need something in the centre.  [SYSTEM] Fitzbillies Restau

generate for b after making changes in sh file

In [16]:
!bash /content/gdrive/MyDrive/nlp/fine_tune_fpt2.sh

03/12/2022 02:48:39 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/12/2022 02:48:39 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_nam

generate for c after making changes in sh file

In [17]:
!bash /content/gdrive/MyDrive/nlp/fine_tune_fpt3.sh

03/12/2022 03:03:53 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/12/2022 03:03:53 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_nam

In [ ]:
def main():
    gen_mode = 0
    gen_labels = ['logits','greedy','beam','top-p', 'top-k', 'temperature']
    tuned_model = 1
    
    if tuned_model == 0:
        tuned = 'gpt2'
        test_name = 'woz.test_a.txt'
    elif tuned_model == 1:
        tuned = '/content/gdrive/MyDrive/nlp/b'
        test_name = 'woz.test_b.txt'
    else:
        tuned = '/content/gdrive/MyDrive/nlp/c'
        test_name = 'woz.test_c.txt'
    
    tokenizer = GPT2Tokenizer.from_pretrained(tuned)
    model = GPT2LMHeadModel.from_pretrained(tuned,pad_token_id=tokenizer.eos_token_id)
    model = model.cuda()
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])        
    print('total parameters = ',params)

    # Metrics
    metric_bleu = load_metric("bleu")
    # NEW METRICS ADDED
    metric_rouge = load_metric("rouge")
    metric_bleurt = load_metric("bleurt")
    metric_wikisplit = load_metric("wiki_split")
    
    predicts = []
    refs = []
    best = []
    bleus = []
    # New metrics lists
    rouges = []
    bleurts = []
    wikisplit = []

    max_len = 0
    total = 0
    with open(test_name,'rt') as f:
        for line in f:
            line = line.replace('\n','')
            text = line.split('|')
            prompt = text[0].strip(' ')
            in_ids = tokenizer.encode(prompt,add_special_tokens=True)
            if len(in_ids) > max_len:
                max_len = len(in_ids)
            total = total + 1
            
    max_len = max_len + 32
    print('max_len: %d total: %d' % (max_len,total))
    
    obs = 0
    with open(test_name,'rt') as f:
        for line in f:
            line = line.replace('\n','')
            text = line.split('|')
            prompt = text[0].strip(' ')
            ref = text[1].strip(' ')
            obs = obs + 1
            in_ids = tokenizer.encode(prompt,add_special_tokens=True)
            
            # Logits
            if gen_mode == 0:
                seq_len = 0
                bDone = False
                while not bDone:
                    input_ids = torch.tensor(in_ids).unsqueeze(0)
                    input_ids = input_ids.cuda()
                    outputs = model(input_ids, labels=input_ids)
                    decoded = []
                    for i in range(outputs[1].size(1)):
                        decoded.append(torch.argmax(outputs[1][0][i][:]).item())
                    decoded = torch.tensor(decoded)
                    decoded = decoded.cuda()
                    in_ids.append(decoded[decoded.size(0)-1].item())
                    input_ids = torch.tensor(in_ids).unsqueeze(0)
                    text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
                    tokens = text.split(' ')
                    if tokens[len(tokens)-1] == '[END]':
                        bDone = True
                    if len(tokens) >= max_len:
                        bDone = True
            # Greedy decoder            
            if gen_mode == 1:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                greedy = model.generate(input_ids, max_length=max_len)  
                text2 = tokenizer.decode(greedy[0], skip_special_tokens=False)
                tokens = text2.split()
            
            # Beam search decoder    
            if gen_mode == 2:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                beam = model.generate(input_ids, max_length=max_len, num_beams = 5, early_stopping = True)  
                text2 = tokenizer.decode(beam[0], skip_special_tokens=False)
                tokens = text2.split()
            
            # Top-p decoder    
            if gen_mode == 3:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                top_p = model.generate(input_ids, max_length=max_len, do_sample = True, top_p = 0.90, top_k = 0)  
                text2 = tokenizer.decode(top_p[0], skip_special_tokens=False)
                tokens = text2.split()
            
            # NEW DECODERS ADDED
            # Top-k decoder
            if gen_mode == 4:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                top_k = model.generate(input_ids, max_length=max_len, do_sample = True, top_k = 50)  
                text2 = tokenizer.decode(top_k[0], skip_special_tokens=False)
                tokens = text2.split()
            
            # Temperature decoder
            if gen_mode == 5:
                input_ids = torch.tensor(in_ids).unsqueeze(0)
                input_ids = input_ids.cuda()
                temp = model.generate(input_ids, max_length=max_len, do_sample = True, top_k = 0, temperature=0.7)  
                text2 = tokenizer.decode(temp[0], skip_special_tokens=False)
                tokens = text2.split()

                
            first = len(prompt.split(' '))
            try:
                pos_end = tokens.index('[END]')
            except:
                pos_end = len(tokens)
            try:
                pos_enduser = tokens.index('[END][USER]')
            except:
                pos_enduser = len(tokens)
            try:
                pos_endsystem = tokens.index('[END][SYSTEM]')
            except:
                pos_endsystem = len(tokens)
            last = min(pos_end,pos_enduser,pos_endsystem,len(tokens))
            predict = ' '.join(tokens[first:last])

            predictions = [predict.split()]
            references = [[ref.split()]]
            predicts.append(predictions)
            refs.append(references)

            try:
                results = metric_bleu.compute(predictions = predictions, references = references)
                bleus.append(results['bleu'])
            except:
                print('ref:  ',ref)
                print('pred: ',predict)
            if results['bleu'] > 0.01:
                print('ref:  ',ref)
                print('pred: ',predict)
                print('BLEU[%d]: %7.3f' % (obs,results['bleu']))
                print(' ')
                best.append(results['bleu'])
    
    print(best)
    if len(best) > 0:
        print('avg[%d]: %7.5f' % (len(best),sum(best)/float(len(best))))
        print(' ')

    results = metric_bleu.compute(predictions = predicts, references = refs)
    print('Final %s on %s BLEU: %7.3f % 7.3f' % (gen_labels[gen_mode],test_name,results['bleu'],sum(bleus)/511.0))
    print(len(predicts),len(refs))
    print(' ')
         
if __name__ == "__main__":
    main()

total parameters =  124439808


Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
max_len: 96 total: 511
ref:   Thank you for using Cambridge TownInfo centre. Good-bye [END]
pred:  Thank you for using our system. Goodbye.
BLEU[7]:   0.309
 
ref:   The address is Cambridge Leisure 